# Live Weapon Detection

The dataset can be downloaded at https://www.kaggle.com/datasets/ankan1998/weapon-detection-dataset.

The data is stored in XML format. Let's use the XMLTree API to extract the features and labels.

In [1]:
import xml.etree.ElementTree as ET
# create a csv: filename, width, height, depth, xmin, ymin, xmax, ymax
text = ''
def write_to_csv(filepath:str, output_path:str) -> None:
    """

    Writes elements from an XML file to a CSV file.

    Parameters:
        filepath (str): Filepath pointing to the XML file
        output_path (str): Name of the CSV file to write

    Returns:
        None

    """
    tree = ET.parse(filepath)
    root = tree.getroot()
        
    # write contents
    text = ''
    for child in root:
        if child.tag == 'filename':
            text += str(child.text)
        if child.tag == 'size':
            # 0 = width, 1 = height, 2 = depth
            for i in range(3):
                text += ',' + str(child[i].text)
        if child.tag == 'object':
            text += ',' + str(child[0].text)
            # 0 = xmin, 1 = ymin, 2 = xmax, 3 = ymax
            for i in range(4):
                text += ',' + str(child[4][i].text)
            # There may be multiple bounding boxes
            # For now, only consider the first one
            break

    with open(output_path, 'a') as csvfile:
        csvfile.write(text)
        csvfile.write('\n')

In [2]:
import os
from skimage.transform import resize
from PIL import Image
from torchvision import transforms

def resize_images(images_path:str, resized_width:int, resized_height:int, save=False) -> tuple:
    """
    
    Resizes images to a given width and height.

    Parameters:
        images_path (str): Filepath containing the images to resize.
        resized_width (int): Width to resize the images.
        resized_height (int): Height to resize the images.

    Returns:
        tuple: NumPy arrays containing the resized images and generated labels.
    
    """
    if save:
        os.makedirs('datasets/images_resized/', exist_ok=True)
    # Resize images
    resized_images = []
    labels = []
    count = 0
    for filename in os.listdir(images_path):
        file_path = os.path.join(images_path, filename)
        if os.path.isfile(file_path):
            with Image.open(file_path) as img:
                # With Scikit-Image:
                # img_data = np.asarray(img)
                # resized = resize(img_data, (resized_width, resized_height))
                # resized_images.append(resized)

                # With PyTorch:
                resize_transform = transforms.Resize((resized_height, resized_width))
                resized = resize_transform(img)

                if save:
                    # Saves resized images
                    resized.save('datasets/images_resized/' + filename)
                    count += 1
                    
                resized_images.append(resized)

    return resized_images

Store the data in CSV format.

In [3]:
with open('weapons.csv', 'a') as csvfile:
    header = 'Filename,Width,Height,Depth,Name,Xmin,Ymin,Xmax,Ymax'
    csvfile.write(header)
    csvfile.write('\n')

with open('weapons_test.csv', 'a') as csvfile:
    header = 'Filename,Width,Height,Depth,Name,Xmin,Ymin,Xmax,Ymax'
    csvfile.write(header)
    csvfile.write('\n')

path = 'datasets/Sohas_weapon-Detection/annotations/xmls/'

for filename in os.listdir(path):
    file_path = os.path.join(path, filename)
    if os.path.isfile(file_path):
        write_to_csv(file_path, 'weapons.csv')

test_path = 'datasets/Sohas_weapon-Detection/annotations_test/xmls/'

for filename in os.listdir(test_path):
    file_path = os.path.join(test_path, filename)
    if os.path.isfile(file_path):
        write_to_csv(file_path, 'weapons_test.csv')

Since the data is separated into two subdirectories, let's combine them and we will split them later.

In [4]:
import pandas as pd
# Get min dimensions
weapons_data = pd.concat(
    map(pd.read_csv, ['weapons.csv', 'weapons_test.csv']), ignore_index=True
)
min_width = weapons_data['Width'].min()
min_height = weapons_data['Height'].min()

In [5]:
# Saves resized images
images_path = 'datasets/images_full'
img = resize_images(images_path, min_width, min_height, save=True)

In [6]:
# Get resized image data
images_path ='datasets/images_resized'
resized_images = resize_images(images_path, min_width, min_height)

## Prepare Image Data for the Model

Split the image dataset into train, test, and valid subdirectories.

In [7]:
import shutil

src = 'datasets/images_resized/'
train_dest = os.path.join(src, 'train')
os.makedirs(train_dest, exist_ok=True)
test_dest = os.path.join(src, 'test')
os.makedirs(test_dest, exist_ok=True)
val_dest = os.path.join(src, 'val')
os.makedirs(val_dest, exist_ok=True)

num_images = 1472
train_percentage = 0.70
test_percentage = 0.20
num_train = int(num_images * train_percentage)
num_test = int(num_images * test_percentage)
num_val = num_images - num_train - num_test

count = 0
for filename in os.listdir(src):
    file_path = os.path.join(src, filename)
    if os.path.isfile(file_path) and count < num_train: 
        # Ensure the file is a file and only move images up to train percentage
        shutil.move(file_path, train_dest)
    count += 1

count = 0
for filename in os.listdir(src):
    file_path = os.path.join(src, filename)
    if os.path.isfile(file_path) and count < num_test: 
        shutil.move(file_path, test_dest)
    count += 1

count = 0
for filename in os.listdir(src):
    file_path = os.path.join(src, filename)
    if os.path.isfile(file_path) and count < num_val:
        shutil.move(file_path, val_dest)
    count += 1

In [9]:
# Create a classes/labels subdirectory that specify if the image has a weapon or not
for filename in os.listdir(train_dest):
    # Get the corresponding image file with the same name
    results = weapons_data[weapons_data['Filename'] == filename]
    # Check if the image contains a weapon
    #print(results['Name'].values)
    

Next, we will convert this image dataset into COCO format using this tutorial: https://medium.com/codable/convert-any-dataset-to-coco-object-detection-format-with-sahi-95349e1fe2b7. 

After that, use Roboflow's RF-DETR object detection model and customize it for this dataset. The docs can be found here: https://rfdetr.roboflow.com/learn/train/#dataset-structure.